In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f
from pyspark.sql.functions import col, signum

spark = SparkSession.builder.master("local[1]").getOrCreate()


In [2]:
#Need to carry over from day to day
#Dollar bars
df=spark.read.parquet("Allcandlestick.parquet")
df.show(5)

+------+-------+------+-------+-------+------+----------+------+
|Ticker|   Open|  High|    Low|  Close|Volume|      Time|Status|
+------+-------+------+-------+-------+------+----------+------+
|   AAT|  35.82| 35.82|  35.82|  35.82| 304.0|1583965200|    ok|
|    AB|  35.88| 35.88|  35.88|  35.88| 100.0|1582235820|    ok|
|  AAAU|  14.89| 14.89|  14.88|  14.88| 868.0|1568210640|    ok|
|   AAP|157.964|158.58|157.964|158.105|6226.0|1573846380|    ok|
|  AACG|   1.28|  1.34|   1.26|   1.34|5370.0|1591882500|    ok|
+------+-------+------+-------+-------+------+----------+------+
only showing top 5 rows



In [3]:
import sys
df=df.withColumn('DolExch',col('Close')*col('Volume'))
df.select("DolExch").distinct().show(5)
df=df.withColumn("CumDolExch",f.sum("DolExch").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
df.select("CumDolExch").distinct().show(5)
df=df.withColumn('DolBars', col('CumDolExch')%(1E7))#Take data pt every 10M dollars exchanged per stock
df=df.withColumn('Mark',f.when(df.DolBars<=df.DolExch,1).otherwise(0))
df.select("Mark").distinct().show()

+-------------+
|      DolExch|
+-------------+
|      231.235|
|  1232713.635|
|   1312601.32|
|1.603866854E7|
|     33301.68|
+-------------+
only showing top 5 rows

+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+
|Ticker| Open| High|  Low|  Close|Volume|      Time|Status|           DolExch|       CumDolExch|
+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+
|   AAT|47.64|47.64|47.64|  47.64|1621.0|1562607000|    ok|          77224.44|         77224.44|
|   AAT| 47.8| 47.8| 47.8|   47.8| 200.0|1562607600|    ok|            9560.0|         86784.44|
|   AAT| 47.8|47.97|47.74|  47.97|3076.0|1562607900|    ok|         147555.72|        234340.16|
|   AAT|47.96|47.97|47.89|47.8925|2097.0|1562607960|    ok|       100430.5725|      334770.7325|
|   AAT|47.89|47.91|47.88|  47.88|1744.0|1562608020|    ok|          83502.72|      418273.4525|
|   AAT|47.83|47.83|47.82|  47.82| 800.0|1562608080|   

In [4]:
df=df.withColumn('CumMark',f.sum("Mark").over(Window.partitionBy('Ticker').orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show(20)
df.select("CumMark").distinct().show(10)

+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+-----------------+----+-------+
|Ticker| Open| High|  Low|  Close|Volume|      Time|Status|           DolExch|       CumDolExch|          DolBars|Mark|CumMark|
+------+-----+-----+-----+-------+------+----------+------+------------------+-----------------+-----------------+----+-------+
|   AAT|47.64|47.64|47.64|  47.64|1621.0|1562607000|    ok|          77224.44|         77224.44|         77224.44|   1|      1|
|   AAT| 47.8| 47.8| 47.8|   47.8| 200.0|1562607600|    ok|            9560.0|         86784.44|         86784.44|   0|      1|
|   AAT| 47.8|47.97|47.74|  47.97|3076.0|1562607900|    ok|         147555.72|        234340.16|        234340.16|   0|      1|
|   AAT|47.96|47.97|47.89|47.8925|2097.0|1562607960|    ok|       100430.5725|      334770.7325|      334770.7325|   0|      1|
|   AAT|47.89|47.91|47.88|  47.88|1744.0|1562608020|    ok|          83502.72|      418273.4525|      41

In [5]:
df=df.withColumn("Volume", f.sum("Volume").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))

df=df.withColumn("Low",f.min("Low").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df.show()
print(df.head())
df=df.withColumn("Open",f.first("Open").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
print(df.head())
df=df.withColumn("Close",f.last("Close").over(Window.partitionBy("CumMark").orderBy("Time").rowsBetween(-sys.maxsize,0)))
df=df.filter(df.Mark==1)

#Add target vector(2 for +change, 1 for no change 0 for -change)
df=df.withColumn("next_val", f.lead(col("Close"),1).over(Window.partitionBy('Ticker').orderBy(df["Time"])))#Need to add partitionBy()
df=df.withColumn("Target", (col("next_val")-col("Close"))/col("Close"))
print(df.head())
df=df.select("Ticker","Time","Open","Low","High","Close","Volume","Target")#.withColumn("Target", 100*(col("next_val")-col("Close"))/col("Close"))
#df=df.drop("next_val").withColumn("Target",1+signum(col("Target")))
df.head()

+------+--------+--------+------+--------+--------+----------+------+------------------+--------------------+------------------+----+-------+
|Ticker|    Open|    High|   Low|   Close|  Volume|      Time|Status|           DolExch|          CumDolExch|           DolBars|Mark|CumMark|
+------+--------+--------+------+--------+--------+----------+------+------------------+--------------------+------------------+----+-------+
|  AAPL|  199.31|  199.31|199.03|199.0433|177630.0|1562608020|    ok|    3.5356061379E7|    8.683384657028E8|8338465.7028000355|   1|     26|
|  ABBV| 71.5335| 71.5529| 71.45|   71.45|260144.0|1562612640|    ok|         5895625.3|2.5500461000989988E8| 5004610.009899884|   1|     26|
|  ABBV| 71.4555|   71.56| 71.42|   71.56|289360.0|1562612700|    ok|        2090696.96| 2.570953069698999E8| 7095306.969899893|   0|     26|
|  ABBV|   71.56|   71.56| 71.42|    71.5|322435.0|1562612760|    ok|         2364862.5| 2.594601694698999E8| 9460169.469899893|   0|     26|
|   AA

Row(Ticker='AAT', Time=1562607000, Open=76.2, Low=4.95, High=47.64, Close=47.64, Volume=789039.0, Target=0.007136859781695976)

In [6]:
#Convert Ticker value to int to include in ML algo, dictionary present (tick_dict) to convert back if needed
tickers=df.select("Ticker").distinct().rdd.flatMap(lambda x: x).collect()

tick_dict = {val : str(idx + 1) for idx, val in enumerate(tickers)} 
print(tick_dict)

from pyspark.sql.types import IntegerType
df=df.replace(to_replace=tick_dict, subset=['Ticker'])

df.printSchema()
df=df.withColumn("Ticker",col("Ticker").cast(IntegerType()))
df.printSchema()

{'AAT': '1', 'ABMD': '2', 'AA': '3', 'AAPL': '4', 'ACA': '5', 'ABM': '6', 'AACG': '7', 'ABEO': '8', 'ABTX': '9', 'ABIO': '10', 'ABEV': '11', 'ABBV': '12', 'AAP': '13', 'AAON': '14', 'AAWW': '15', 'ABUS': '16', 'ABG': '17', 'AAXN': '18', 'ACAM': '19', 'ABCB': '20', 'AAOI': '21', 'A': '22', 'ABEQ': '23', 'AAAU': '24', 'AAL': '25', 'ABR': '26', 'AAXJ': '27', 'AB': '28', 'AAU': '29', 'ABT': '30', 'ABB': '31', 'ABC': '32', 'AC': '33', 'ACBI': '34', 'AAME': '35', 'ACAD': '36', 'ACAMU': '37', 'ACB': '38', 'AADR': '39', 'ACAMW': '40'}
root
 |-- Ticker: string (nullable = true)
 |-- Time: long (nullable = true)
 |-- Open: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Target: double (nullable = true)

root
 |-- Ticker: integer (nullable = true)
 |-- Time: long (nullable = true)
 |-- Open: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- High: double (n

In [7]:
from pyspark.ml.feature import VectorAssembler
#Create input features
df_cols=df.columns
df_cols=[ elem for elem in df_cols if elem not in ["Time","Target"]]
#Move features to a single vector
assembler=VectorAssembler(inputCols=df_cols,outputCol="features")
df=assembler.transform(df)
df=df.na.drop()
df=df.dropDuplicates()
print(df.count())

152960


In [8]:
df.show(10)
df=df.na.drop()
df=df.dropDuplicates()

+------+----------+--------+-----+--------+-------+---------+--------------------+--------------------+
|Ticker|      Time|    Open|  Low|    High|  Close|   Volume|              Target|            features|
+------+----------+--------+-----+--------+-------+---------+--------------------+--------------------+
|     1|1581386520|  199.92| 5.21|   46.15|  46.15|6429415.0|                 0.0|[1.0,199.92,5.21,...|
|     1|1585596780|  199.93| 5.27|   24.23|  24.23|7589037.0|0.022286421791167937|[1.0,199.93,5.27,...|
|     1|1591048140|199.8948| 5.12|   27.02|   27.0|7875015.0| 7.40740740740725E-4|[1.0,199.8948,5.1...|
|     2|1564689120|  199.49| 5.38| 208.349| 207.77|2578812.0|-0.00514992539827...|[2.0,199.49,5.38,...|
|     2|1566858780| 201.035| 4.47|  190.12| 190.12|1822585.0|0.001735745844729...|[2.0,201.035,4.47...|
|     2|1570049880|  203.04| 4.66|  165.25|  165.0|2923004.0|-0.00515151515151...|[2.0,203.04,4.66,...|
|     2|1570562640| 203.185|4.665|  160.08| 160.08|2646993.0|0.0

In [9]:
from pyspark.ml.regression  import RandomForestRegressor
import time




train,test=df.randomSplit([0.8,0.2],seed=1)
train.show(10)
print('repartitioning')
train=train.repartition(10)
test=test.repartition(10)
print('End repartition')

start_time=time.time()

dt=RandomForestRegressor(featuresCol='features',
                         labelCol='Target',
                         maxDepth=30,
                         minInstancesPerNode=2)
print('Start training')
dtModel=dt.fit(train)
print('End training')
predictions=dtModel.transform(test)
end_time=time.time()

delta_time = end_time - start_time

# 5. print total run time 
print(f'run-time: {round(delta_time, 2)}')

predictions.printSchema()

#predictions.select('Time','Target','probability','prediction').show()

+------+----------+--------+------+--------+-------+---------+--------------------+--------------------+
|Ticker|      Time|    Open|   Low|    High|  Close|   Volume|              Target|            features|
+------+----------+--------+------+--------+-------+---------+--------------------+--------------------+
|     1|1581386520|  199.92|  5.21|   46.15|  46.15|6429415.0|                 0.0|[1.0,199.92,5.21,...|
|     1|1585596780|  199.93|  5.27|   24.23|  24.23|7589037.0|0.022286421791167937|[1.0,199.93,5.27,...|
|     1|1591048140|199.8948|  5.12|   27.02|   27.0|7875015.0| 7.40740740740725E-4|[1.0,199.8948,5.1...|
|     2|1564689120|  199.49|  5.38| 208.349| 207.77|2578812.0|-0.00514992539827...|[2.0,199.49,5.38,...|
|     2|1570049880|  203.04|  4.66|  165.25|  165.0|2923004.0|-0.00515151515151...|[2.0,203.04,4.66,...|
|     2|1570562640| 203.185| 4.665|  160.08| 160.08|2646993.0|0.002623688155921...|[2.0,203.185,4.66...|
|     2|1571873940|  203.94| 4.505|  178.75| 178.72|279

In [10]:
#from pyspark.ml.tuning import CrossValidator


In [11]:
max=predictions.agg({"prediction":"max"}).collect()[0]
max=max["max(prediction)"]#Max predicted increase

toDisplay=predictions.select("Time","Target","prediction")
toDisplay=toDisplay.withColumn("ToInvest",f.when(col('prediction')>0,1000.*col("prediction")/max).otherwise(0))
toDisplay=toDisplay.withColumn("Revenue", col("ToInvest")*col("Target"))
#toDisplay=toDisplay.withColumn("Target",f.round(100*col("Target"),5))
#toDisplay=toDisplay.withColumn("prediction",f.round(100*col("prediction"),5))
toDisplay.show()
print(predictions.count())

+----------+--------------------+--------------------+--------------------+--------------------+
|      Time|              Target|          prediction|            ToInvest|             Revenue|
+----------+--------------------+--------------------+--------------------+--------------------+
|1567704900|0.011451971127151423|3.456266235097209E-4|  0.5975054084404046|0.006842614685776331|
|1580932740|0.006048621612190253|4.794524063720834E-5| 0.08288580404716109|5.013448657034249E-4|
|1563404820|0.001383672088589...|-1.50338051465750...|                 0.0|                 0.0|
|1563905100|3.599366511493431...|-4.15745063039012...|                 0.0|                 0.0|
|1572983220|5.835667600372598E-4|-1.79519234424340...|                 0.0|                 0.0|
|1589840760|2.740030480060778...|-1.27425999260278...|                 0.0|                 0.0|
|1584458940|6.013229104029804E-4|-1.01366015652250...|                 0.0|                 0.0|
|1580514420|3.511977191747271E

In [ ]:
profit_row=toDisplay.agg({"Revenue":"sum"}).collect()[0]

profit=profit_row["sum(Revenue)"]
print(profit)

toDisplay.filter(col("ToInvest")>0).show()

-39.21212737325498
